In [ ]:
import sys
# TODO: why is this necessary, I am running from dmt/v2, shouldn't it be in path?
sys.path.append("/home/dictus/dmt/")
from dmt.analysis import StructuredAnalysis

In [ ]:
import pandas as pd
from neuro_dmt.terminology import circuit, cell, neuroscience
from dmt.tk.terminology.data import data as data
from dmt.tk.terminology import Term
CellDensity = Term("cell density", "number of cells per mm^3")


In [ ]:

from neuro_dmt.models.bluebrain.circuit.adapter import CircuitAdapter
from neuro_dmt.models.bluebrain.circuit.model import CircuitModel

In [ ]:
import pandas as pd
by_layer_example_cell_density = pd.DataFrame(
        {circuit.layer: ['L1',] * 3 + ['L2',] * 3 + ['L3',] * 3 + ['L4',] * 3 + ['L5',] * 3 + ['L6',] * 3,
         CellDensity: [100000, 110000, 90000,
                       120000, 130000, 140000,
                       110000, 200000, 60000,
                       130000, 130000, 130000,
                       100000, 130000, 160000,
                       231000, 214000, 121000]})

In [ ]:
# measurement_parameters should (in principle) never touch the parameters, just pass them through its methods. Is it worthwhile to enforce that? (I think: No, if people need to do it for a quickyly-hacked solution let them, and it will be caught by code review before it hits the main library)
# TODO: adapter methods are coupled to the measurement in part, we need a way to aggregate the Interface from the Adapter's components
#       it may be good to combine this with a check that runs the measurement/parameters object's __call__ on its 'adapter' object with mock model and parameters, and assert that a NotImplementedError is raised
from dmt.tk.reporting import Reporter

def cell_density(adapter, model, **parameters):
    """get cell density for a model through its adapter"""
    return adapter.get_cell_density(model, **parameters)
cell_density.phenomenon = CellDensity

from dmt.tk.plotting.bars import Bars

class AdapterInterface:
    def get_cell_density(self, model, **params):
        raise NotImplementedError()
        
DataReplicationCellDensityAnalysis = StructuredAnalysis(measurement_method=cell_density,
                                             plotter=Bars(),AdapterInterface=AdapterInterface, 
                                                          default_adapter=CircuitAdapter(),
                                                       reporter=Reporter(path_output_folder='/home/dictus/validations'))


In [ ]:
circuit_config = "/gpfs/bbp.cscs.ch/project/proj68/circuits/O1/20190307/CircuitConfig"
circuit_model = CircuitModel(circuit_config)
example_report = DataReplicationCellDensityAnalysis.with_fields(reference_data=by_layer_example_cell_density)((CircuitAdapter(), circuit_model))


In [ ]:
MC_by_layer_data = pd.DataFrame({
    neuroscience.layer: ['L2', 'L3', 'L4', 'L6'],
   cell.morphology_subclass: 'MC',
   CellDensity: [12000, 10000, 20000, 10000]
})
MC_by_layer_report = DataReplicationCellDensityAnalysis.with_fields(reference_data=MC_by_layer_data)(circuit_model)

In [ ]:
from neuro_dmt.models.bluebrain.circuit.atlas import CircuitAtlas

thalamus_data = pd.DataFrame({
    CellDensity : [72802.84, 65364.87, 56678.06, 75391.47],
    circuit.region: ['RT', '@VPL|VPM', 'VPL', 'VPM']})
thalamus_atlas = CircuitAtlas("/gpfs/bbp.cscs.ch/project/proj55/entities/atlas/20190628/")
thalamus_atlas.label = "Thalamus Atlas"
# using the CircuitAdapter is wierd.
# Maybe just call it 'BlueBrainAdapter'
DataReplicationCellDensityAnalysis.with_fields(reference_data=thalamus_data)((CircuitAdapter(), thalamus_atlas))

In [ ]:
import yaml
DefilipeDatapath = "/gpfs/bbp.cscs.ch/project/proj64/dissemination/data/DeFelipe2017.yaml"
with open(DefilipeDatapath) as DFDF:
    DFDict = yaml.load(DFDF)
print(DFDict)
import pandas as pd
DefilipeCellDensityData = pd.DataFrame({circuit.layer: "L{}".format(l+1),
                                        CellDensity: subj['densities'][l]}
                                        for l in range(0,6)
                                        for subj in DFDict['circuits'].values())
print(DefilipeCellDensityData)

In [ ]:
rat = "mean"
ratlas = CircuitAtlas("/gpfs/bbp.cscs.ch/project/proj64/dissemination/data/atlas/O1/DAN/"+rat)
DataReplicationCellDensityAnalysis.with_fields(reference_data=DefilipeCellDensityData)(ratlas)

In [ ]:
S1_atlas = CircuitAtlas("/gpfs/bbp.cscs.ch/project/proj64/dissemination/data/atlas/S1/MEAN/juvenile_L23_MC_BTC_shifted_down/Bio_M/")
S1_circuit = CircuitModel("/gpfs/bbp.cscs.ch/project/proj64/dissemination/circuits/S1/juvenile/L23_MC_BTC_shifted_down/Bio_M/20190925/CircuitConfig")
S1_atlas.label = 'Bio_M Atlas'
S1_circuit.label = "Bio_M Circuit"
report = DataReplicationCellDensityAnalysis.with_fields(
    measurement_parameters=
    pd.DataFrame({neuroscience.layer: ['L1', 'L2', 'L3', 'L4', 'L5', 'L6'],
                  neuroscience.region: [['SSp-ul', 'SSp-ll', 'SSp-sh', 'SSp-tr']]*6,
                  neuroscience.synapse_class: 'EXC'}))(S1_atlas, S1_circuit)

In [ ]:
report.measurement